In [1]:
%pip install pandas > /dev/null

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from os import listdir

DATA_DIR = 'data/raw/school-scores/'
data_paths = [f'{DATA_DIR}{f}' for f in listdir(DATA_DIR)]

In [2]:
COLUMNS_TO_SELECT = [
    'RSPO', 'Kod teryt gminy', 'Typ gminy',
    'liczba\nzdających', 'wynik\nśredni (%)',
    'liczba\nzdających.1', 'wynik\nśredni (%).1',
    'liczba\nzdających.2', 'wynik\nśredni (%).2',
]

In [3]:
data = []
for path in data_paths:
    df = pd.read_csv(path, skiprows=1, dtype={'RSPO': str, 'Kod teryt gminy': str})
    df = df[(df['rodzaj placówki\n'] == 'dla młodzieży') & (df['czy publiczna'] == 'Tak')]
    df = df[COLUMNS_TO_SELECT]
    df.dropna(thresh=6, inplace=True)
    data.append(df)

data = pd.concat(data)

In [4]:
COUNTY_TYPE = 'Typ gminy'
county_types = data[COUNTY_TYPE].value_counts().keys().tolist()
for county_type in county_types:
    data[county_type] = (data[COUNTY_TYPE] == county_type).astype(int)

In [5]:
DATA_PATH = 'data/raw/counties/revenue-per-person.csv'
revenue_per_person = pd.read_csv(DATA_PATH, sep=';', dtype={'TERYT': str})
data = data.merge(revenue_per_person, left_on='Kod teryt gminy', right_on='TERYT')
data.drop(columns=['Kod teryt gminy', COUNTY_TYPE, 'Name'], inplace=True)

In [6]:
COLUMNS_TO_SELECT = [
    "Numer RSPO",
    "Strona www",
    "Data rozpoczęcia działalności",
    # "Miejsce w strukturze",
    "Liczba uczniów",
    "Tereny sportowe",
    "Języki nauczane",
    "Czy zatrudnia logopedę",
    "Czy zatrudnia psychologa",
    "Czy zatrudnia pedagoga",
]

In [7]:
rspo = pd.read_csv(f'data/raw/schools/rspo.csv', sep=';', dtype={"Numer RSPO": str})
rspo = rspo[COLUMNS_TO_SELECT]
rspo.dropna(subset=["Liczba uczniów"], inplace=True)
rspo["Strona www"] = rspo["Strona www"] != "http://"
rspo["Tereny sportowe"] = rspo["Tereny sportowe"].notna()
rspo["Języki nauczane"] = rspo["Języki nauczane"].str.contains("niemiecki|francuski")
rspo["Czy zatrudnia logopedę"] = rspo["Czy zatrudnia logopedę"] == "TAK"
rspo["Czy zatrudnia psychologa"] = rspo["Czy zatrudnia psychologa"] == "TAK"
rspo["Czy zatrudnia pedagoga"] = rspo["Czy zatrudnia pedagoga"] == "TAK"
rspo = rspo.astype({"Języki nauczane": bool})
rspo = rspo.astype({
    'Liczba uczniów': int, "Strona www": int, "Tereny sportowe": int, "Języki nauczane": int,
    "Czy zatrudnia logopedę": int, "Czy zatrudnia psychologa": int, "Czy zatrudnia pedagoga": int,
})

In [8]:
data = data.merge(rspo, left_on='RSPO', right_on='Numer RSPO')
data.drop(columns=['Numer RSPO'], inplace=True)

In [9]:
data.rename(columns={
    'liczba\nzdających': 'Participants in Polish',
    'wynik\nśredni (%)': 'Avg. score in Polish',
    'liczba\nzdających.1': 'Participants in mathematics',
    'wynik\nśredni (%).1': 'Avg. score in mathematics',
    'liczba\nzdających.2': 'Participants in English',
    'wynik\nśredni (%).2': 'Avg. score in English',
    'Gmina wiejska': 'Rural county',
    'Gmina miejska': 'Urban county',
    'Obszar wiejski': 'Rural territory',
    'Miasto': 'City',
    "Strona www": "Website",
    "Data rozpoczęcia działalności": "Start date",
    "Liczba uczniów": "Students",
    "Tereny sportowe": "Sport venues",
    "Języki nauczane": "German or French",
    "Czy zatrudnia logopedę": "Logopedist",
    "Czy zatrudnia psychologa": "Psychologist",
    "Czy zatrudnia pedagoga": "Pedagogue",
}, inplace=True)

COLUMNS_ORDER=[
    'RSPO', 'TERYT',
    'Rural county', 'Urban county', 'Rural territory', 'City',
    'Revenue per person',
    "Website", "Start date", "Students", "Sport venues", "German or French",
    "Logopedist", "Psychologist", "Pedagogue",
    'Participants in Polish', 'Avg. score in Polish',
    'Participants in mathematics', 'Avg. score in mathematics',
    'Participants in English', 'Avg. score in English',
]
data = data[COLUMNS_ORDER]

In [10]:
polish_data = data.drop(columns=[
  'Participants in mathematics',
  'Avg. score in mathematics',
  'Participants in English',
  'Avg. score in English',
]).dropna().astype({
  'Participants in Polish': int,
  'Avg. score in Polish': int,
})

mathematics_data = data.drop(columns=[
  'Participants in Polish',
  'Avg. score in Polish',
  'Participants in English',
  'Avg. score in English',
]).dropna().astype({
  'Participants in mathematics': int,
  'Avg. score in mathematics': int,
})

english_data = data.drop(columns=[
  'Participants in Polish',
  'Avg. score in Polish',
  'Participants in mathematics',
  'Avg. score in mathematics',
]).dropna().astype({
  'Participants in English': int,
  'Avg. score in English': int,
})

In [11]:
exams_data = [polish_data, mathematics_data, english_data]
exams = ['polish', 'mathematics', 'english']
for exam_data, exam in zip(exams_data, exams):
    exam_data.to_csv(f'data/combined/{exam}.csv', index=None)